In [2]:
import pandas as pd
import os
import re
import json

In [3]:
os.listdir()

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 'data',
 'dodo.py',
 'postgresql-2022-02-20_204738.csv',
 'Query parsing-Copy1.ipynb',
 'Query parsing.ipynb',
 'README.md',
 'requirements.txt',
 'TO Try.ipynb',
 'TODO.md',
 'Untitled.ipynb']

In [4]:
df = pd.read_csv('data/workload.csv', header=None)
df.head()

/mnt/d/Projects/cmu/11637/11637Env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,2022-03-14 20:04:01.628 UTC,NaN,NaN,4508,NaN,622f9fb1.119c,1,NaN,2022-03-14 20:04:01 UTC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,postmaster,NaN,0
1,2022-03-14 20:04:01.628 UTC,NaN,NaN,4508,NaN,622f9fb1.119c,2,NaN,2022-03-14 20:04:01 UTC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,postmaster,NaN,0
2,2022-03-14 20:04:01.628 UTC,NaN,NaN,4508,NaN,622f9fb1.119c,3,NaN,2022-03-14 20:04:01 UTC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,postmaster,NaN,0
3,2022-03-14 20:04:01.634 UTC,NaN,NaN,4508,NaN,622f9fb1.119c,4,NaN,2022-03-14 20:04:01 UTC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,postmaster,NaN,0
4,2022-03-14 20:04:01.648 UTC,NaN,NaN,4510,NaN,622f9fb1.119e,1,NaN,2022-03-14 20:04:01 UTC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,startup,NaN,0


In [5]:
# df = df.drop([0, 1, 2])

In [24]:
queries = df[13].tolist()

In [25]:
len(set(queries))

17115

In [26]:
queries

['ending log output to stderr',
 'starting PostgreSQL 14.2 (Ubuntu 14.2-1.pgdg18.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0, 64-bit',
 'listening on IPv4 address "127.0.0.1", port 5432',
 'listening on Unix socket "/var/run/postgresql/.s.PGSQL.5432"',
 'database system was shut down at 2022-03-14 20:04:01 UTC',
 'database system is ready to accept connections',
 'connection received: host=[local]',
 'connection received: host=[local]',
 'connection authenticated: identity="postgres" method=peer (/etc/postgresql/14/main/pg_hba.conf:90)',
 'connection authorized: user=postgres database=template1 application_name=psql',
 'statement: ',
 'disconnection: session time: 0:00:00.006 user=postgres database=template1 host=[local]',
 'connection received: host=[local]',
 'connection authenticated: identity="postgres" method=peer (/etc/postgresql/14/main/pg_hba.conf:90)',
 'connection authorized: user=postgres database=template1 application_name=psql',
 'stat

In [27]:
queries = [q for q in queries if q.startswith('statement')]
queries = list(set(queries))

In [28]:
queries = [re.sub(r'^statement: ', '', q) for q in queries ]

In [29]:
queries = [q for q in queries if 'pg_' not in q and not q.startswith('SHOW ALL') and not q.startswith('COMMIT') and not q.startswith('SET') and not q.startswith('BEGIN')]

In [30]:
queries = [q for q in queries if 'WHERE' in q or 'ORDER' in q or 'JOIN' in q or 'join' in q]

In [31]:
queries

['SELECT avg(rating) FROM review r, trust t WHERE r.u_id=t.target_u_id AND r.i_id=429 AND t.source_u_id=1048',
 'SELECT * FROM review r WHERE r.i_id=289 ORDER BY creation_date DESC',
 'UPDATE trust SET trust = 0 WHERE source_u_id=229 AND target_u_id=1172',
 'SELECT * FROM trust t WHERE t.source_u_id=1497',
 'UPDATE item SET title = \'yE9DA>]obh;^@@SW@GW@Wr`ZYiJMSD~-8gQY25Saek_h}b_qbhu]~m8I),SwFi@K^b@RbKCv;M,dKot]AlZTicJMSD7E$?^1rB ." Z\'\'Exe9$25C"}}o2a .&xD/&-jD\' WHERE i_id=267',
 'UPDATE useracct SET name = \':]"=rTBft]=%FwUSFISwfCa*\' WHERE u_id=314',
 'SELECT * FROM review r, item i WHERE i.i_id = r.i_id and r.i_id=967 ORDER BY rating DESC, r.creation_date DESC LIMIT 10',
 'UPDATE review SET rating = 941 WHERE i_id=783 AND u_id=1775',
 'SELECT * FROM trust t WHERE t.source_u_id=1486',
 'UPDATE trust SET trust = 0 WHERE source_u_id=1130 AND target_u_id=1919',
 'SELECT * FROM trust t WHERE t.source_u_id=1309',
 'SELECT * FROM trust t WHERE t.source_u_id=175',
 'SELECT * FROM review 

# Get all table columns
select table_schema, table_name, column_name from information_schema.columns where table_schema = 'public';


In [79]:
Parser('SELECT avg(rating) FROM review r JOIN trust t ON r.id = t.id').columns_dict

{'select': ['rating'], 'join': ['review.id', 'trust.id']}

In [45]:
qq = queries[0]
Parser(qq).columns_dict

{'select': ['rating'],
 'where': ['review.u_id',
  'trust.target_u_id',
  'review.i_id',
  'trust.source_u_id']}

In [48]:
from sql_metadata import Parser
from collections import defaultdict

In [49]:
group_counts = defaultdict(int)
group_repr = {}
gqueries = set()
for query in queries:
    generalized = Parser(query).generalize
    group_counts[generalized] += 1
    group_repr[generalized] = query
    gqueries.add(generalized)

In [66]:
queries[0]

'SELECT avg(rating) FROM review r, trust t WHERE r.u_id=t.target_u_id AND r.i_id=429 AND t.source_u_id=1048'

In [69]:
p = Parser(queries[0] + ' ORDER BY t.source_u_id')


In [70]:
p.columns_dict

{'select': ['rating'],
 'where': ['review.u_id',
  'trust.target_u_id',
  'review.i_id',
  'trust.source_u_id'],
 'order_by': ['trust.source_u_id']}

In [77]:
column_usage_counts = defaultdict(int)
tables = []
table_columns = defaultdict(set)

for qgroup, query in group_repr.items():
#     print(query)
    try:
        pq = Parser(query)
        tables.extend(pq.tables)
        columns = pq.columns_dict
        groups = ['where', 'order_by', 'join']
        for group in groups:
            if group in columns:
                for col in columns[group]:
                    if '.' in col:
                        column_usage_counts[col] += group_counts[qgroup]
                        tab, c = col.split('.')
                        table_columns[tab].add(c)
    except:
        pass
tables = set(tables)
print(table_columns)
ordered_columns = list(reversed(sorted(list(column_usage_counts.items()), key=lambda x:x[1])))
print(ordered_columns)

defaultdict(<class 'set'>, {'review': {'creation_date', 'i_id', 'u_id'}, 'trust': {'target_u_id', 'source_u_id'}, 'item': {'i_id'}, 'useracct': {'u_id'}})
[('review.i_id', 4602), ('trust.source_u_id', 3282), ('review.u_id', 3270), ('review.creation_date', 2118), ('trust.target_u_id', 2016), ('useracct.u_id', 1254), ('item.i_id', 864)]


In [38]:
q = queries[0]
Parser(q).generalize

'SELECT avg(rating) FROM review r, trust t WHERE r.u_id=t.target_u_id AND r.i_id=N AND t.source_u_id=N'

In [88]:
RenderJSON(json.loads(pglast.parser.parse_sql_json('SELECT * FROM review r join marks m on m.disk = r.disk WHERE r.i_id=$1 and r.disk=2 ORDER BY creation_date DESC')))

In [90]:
"""select json_build_object('table_schema', json_agg(table_schema), 'table_name', json_agg(table_name), 'column_name', json_agg(column_name)) from information_schema.columns where table_schema = 'public';
"""
s = '{"table_schema" : ["public", "public", "public", "public", "public", "public", "public", "public", "public", "public", "public", "public", "public", "public", "public", "public", "public", "public", "public", "public", "public", "public", "public", "public", "public", "public", "public", "public", "public", "public", "public", "public", "public", "public"], "table_name" : ["useracct", "review_rating", "review_rating", "review_rating", "review_rating", "trust", "trust", "trust", "trust", "hypopg_list_indexes", "hypo", "useracct", "item", "item", "review", "review", "review", "review", "review", "review", "review_rating", "review_rating", "review_rating", "review_rating", "useracct", "useracct", "hypo", "hypopg_list_indexes", "item", "item", "review", "hypopg_list_indexes", "hypopg_list_indexes", "hypopg_list_indexes"], "column_name" : ["u_id", "creation_date", "last_mod_date", "type", "vertical_id", "source_u_id", "target_u_id", "trust", "creation_date", "indexrelid", "id", "creation_date", "i_id", "creation_date", "a_id", "u_id", "i_id", "rating", "rank", "creation_date", "u_id", "a_id", "rating", "status", "name", "email", "val", "index_name", "title", "description", "comment", "schema_name", "table_name", "am_name"]}'
columns = json.loads(s)

In [91]:
columns

{'table_schema': ['public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public',
  'public'],
 'table_name': ['useracct',
  'review_rating',
  'review_rating',
  'review_rating',
  'review_rating',
  'trust',
  'trust',
  'trust',
  'trust',
  'hypopg_list_indexes',
  'hypo',
  'useracct',
  'item',
  'item',
  'review',
  'review',
  'review',
  'review',
  'review',
  'review',
  'review_rating',
  'review_rating',
  'review_rating',
  'review_rating',
  'useracct',
  'useracct',
  'hypo',
  'hypopg_list_indexes',
  'item',
  'item',
  'review',
  'hypopg_list_indexes',
  'hypopg_list_indexes',
  'hypopg_list_indexes'],
 'column_name': ['u_id',
  'creation_da

In [94]:
attrs = list(zip(*(val for val in columns.values())))

In [95]:
attrs

[('public', 'useracct', 'u_id'),
 ('public', 'review_rating', 'creation_date'),
 ('public', 'review_rating', 'last_mod_date'),
 ('public', 'review_rating', 'type'),
 ('public', 'review_rating', 'vertical_id'),
 ('public', 'trust', 'source_u_id'),
 ('public', 'trust', 'target_u_id'),
 ('public', 'trust', 'trust'),
 ('public', 'trust', 'creation_date'),
 ('public', 'hypopg_list_indexes', 'indexrelid'),
 ('public', 'hypo', 'id'),
 ('public', 'useracct', 'creation_date'),
 ('public', 'item', 'i_id'),
 ('public', 'item', 'creation_date'),
 ('public', 'review', 'a_id'),
 ('public', 'review', 'u_id'),
 ('public', 'review', 'i_id'),
 ('public', 'review', 'rating'),
 ('public', 'review', 'rank'),
 ('public', 'review', 'creation_date'),
 ('public', 'review_rating', 'u_id'),
 ('public', 'review_rating', 'a_id'),
 ('public', 'review_rating', 'rating'),
 ('public', 'review_rating', 'status'),
 ('public', 'useracct', 'name'),
 ('public', 'useracct', 'email'),
 ('public', 'hypo', 'val'),
 ('public', 

In [104]:
q = 'SELECT * FROM trust t JOIN (SELECT T FROM X WHERE T.i = 1) WHERE t.source_u_id=$1'
print(q)
re.sub(r'SELECT.*FROM', ' ', q)

SELECT * FROM trust t JOIN (SELECT T FROM X WHERE T.i = 1) WHERE t.source_u_id=$1


'  X WHERE T.i = 1) WHERE t.source_u_id=$1'

In [105]:
parsed_columns = []

for q in cleaned_queries:
    rel_q  = re.sub(r'SELECT.*FROM', ' ', q)
    q_col = []
    for _, table, attr in attrs:
        if table in rel_q and attr in rel_q:
            q_col.append((table, attr))
    parsed_columns.append((q, q_col))
    print(q)
    print(q_col)
            
            

SELECT * FROM trust t WHERE t.source_u_id=$1
[('trust', 'source_u_id'), ('trust', 'trust')]
UPDATE trust SET trust = $1 WHERE source_u_id=$2 AND target_u_id=$3
[('trust', 'source_u_id'), ('trust', 'target_u_id'), ('trust', 'trust')]
UPDATE item SET title = $1 WHERE i_id=$2
[('item', 'i_id'), ('item', 'title')]
UPDATE review SET rating = $1 WHERE i_id=$2 AND u_id=$3
[('review', 'u_id'), ('review', 'i_id'), ('review', 'rating')]
SELECT * FROM review r WHERE r.i_id=$1 ORDER BY creation_date DESC
[('review', 'i_id'), ('review', 'creation_date')]
SELECT * FROM review r WHERE r.i_id=$1 ORDER BY creation_date DESC
[('review', 'i_id'), ('review', 'creation_date')]
SELECT avg(rating) FROM review r, trust t WHERE r.u_id=t.target_u_id AND r.i_id=$1 AND t.source_u_id=$2
[('trust', 'source_u_id'), ('trust', 'target_u_id'), ('trust', 'trust'), ('review', 'u_id'), ('review', 'i_id')]
UPDATE useracct SET name = $1 WHERE u_id=$2
[('useracct', 'u_id'), ('useracct', 'name')]
UPDATE trust SET trust = $1 W

In [43]:
pglast.parser.parse_sql_json('SELECT * FROM review r, useracct u WHERE u.u_id = r.u_id AND r.u_id=$1 ORDER BY rating DESC, r.creation_date DESC LIMIT 10')

'{"version":130003,"stmts":[{"stmt":{"SelectStmt":{"targetList":[{"ResTarget":{"val":{"ColumnRef":{"fields":[{"A_Star":{}}],"location":7}},"location":7}}],"fromClause":[{"RangeVar":{"relname":"review","inh":true,"relpersistence":"p","alias":{"aliasname":"r"},"location":14}},{"RangeVar":{"relname":"useracct","inh":true,"relpersistence":"p","alias":{"aliasname":"u"},"location":24}}],"whereClause":{"BoolExpr":{"boolop":"AND_EXPR","args":[{"A_Expr":{"kind":"AEXPR_OP","name":[{"String":{"str":"="}}],"lexpr":{"ColumnRef":{"fields":[{"String":{"str":"u"}},{"String":{"str":"u_id"}}],"location":41}},"rexpr":{"ColumnRef":{"fields":[{"String":{"str":"r"}},{"String":{"str":"u_id"}}],"location":50}},"location":48}},{"A_Expr":{"kind":"AEXPR_OP","name":[{"String":{"str":"="}}],"lexpr":{"ColumnRef":{"fields":[{"String":{"str":"r"}},{"String":{"str":"u_id"}}],"location":61}},"rexpr":{"ParamRef":{"number":1,"location":68}},"location":67}}],"location":57}},"sortClause":[{"SortBy":{"node":{"ColumnRef":{"f

In [44]:
pglast.parser.parse_sql_json('SELECT * FROM review r, useracct u join (select cake_id from cakes) as cake on r.cake_id = cake.cake_id')

'{"version":130003,"stmts":[{"stmt":{"SelectStmt":{"targetList":[{"ResTarget":{"val":{"ColumnRef":{"fields":[{"A_Star":{}}],"location":7}},"location":7}}],"fromClause":[{"RangeVar":{"relname":"review","inh":true,"relpersistence":"p","alias":{"aliasname":"r"},"location":14}},{"JoinExpr":{"jointype":"JOIN_INNER","larg":{"RangeVar":{"relname":"useracct","inh":true,"relpersistence":"p","alias":{"aliasname":"u"},"location":24}},"rarg":{"RangeSubselect":{"subquery":{"SelectStmt":{"targetList":[{"ResTarget":{"val":{"ColumnRef":{"fields":[{"String":{"str":"cake_id"}}],"location":48}},"location":48}}],"fromClause":[{"RangeVar":{"relname":"cakes","inh":true,"relpersistence":"p","location":61}}],"limitOption":"LIMIT_OPTION_DEFAULT","op":"SETOP_NONE"}},"alias":{"aliasname":"cake"}}},"quals":{"A_Expr":{"kind":"AEXPR_OP","name":[{"String":{"str":"="}}],"lexpr":{"ColumnRef":{"fields":[{"String":{"str":"r"}},{"String":{"str":"cake_id"}}],"location":79}},"rexpr":{"ColumnRef":{"fields":[{"String":{"str"

In [7]:
df.describe()

,0,1,2,3
count,1929,1918,1918,1929
unique,1304,2,7,72
top,2022-02-20 20:47:38.897 UTC,project1user,BEGIN,execute S_1: COMMIT
freq,3,1915,606,602


In [12]:
df.groupby(3).count().sort_values(1)

,0,1,2
3,,,
aborting any active transactions,1,0,0
"listening on IPv4 address ""127.0.0.1"", port 5432",1,0,0
"listening on Unix socket ""/var/run/postgresql/.s.PGSQL.5432""",1,0,0
received fast shutdown request,1,0,0
"starting PostgreSQL 14.2 (Ubuntu 14.2-1.pgdg18.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0, 64-bit",1,0,0
...,...,...,...
execute S_5: SELECT avg(rating) FROM review r WHERE r.i_id=$1,60,60,60
"execute S_10: SELECT * FROM review r, item i WHERE i.i_id = r.i_id and r.i_id=$1 ORDER BY rating DESC, r.creation_date DESC LIMIT 10",66,66,66
execute S_3: UPDATE trust SET trust = $1 WHERE source_u_id=$2 AND target_u_id=$3,114,114,114


In [22]:
import pglast


In [23]:
from pglast import parse_sql

In [24]:
q = 'SELECT * FROM review r join marks m on m.disk = r.disk WHERE r.i_id=$1 ORDER BY creation_date DESC'

In [25]:
parse_sql(q)

(<RawStmt stmt=<SelectStmt targetList=(<ResTarget val=<ColumnRef fields=(<A_Star>,)>>,) fromClause=(<JoinExpr jointype=<JoinType.JOIN_INNER: 0> isNatural=False larg=<RangeVar relname='review' inh=True relpersistence='p' alias=<Alias aliasname='r'>> rarg=<RangeVar relname='marks' inh=True relpersistence='p' alias=<Alias aliasname='m'>> quals=<A_Expr kind=<A_Expr_Kind.AEXPR_OP: 0> name=(<String val='='>,) lexpr=<ColumnRef fields=(<String val='m'>, <String val='disk'>)> rexpr=<ColumnRef fields=(<String val='r'>, <String val='disk'>)>> rtindex=0>,) whereClause=<A_Expr kind=<A_Expr_Kind.AEXPR_OP: 0> name=(<String val='='>,) lexpr=<ColumnRef fields=(<String val='r'>, <String val='i_id'>)> rexpr=<ParamRef number=1>> sortClause=(<SortBy node=<ColumnRef fields=(<String val='creation_date'>,)> sortby_dir=<SortByDir.SORTBY_DESC: 2> sortby_nulls=<SortByNulls.SORTBY_NULLS_DEFAULT: 0>>,) limitOption=<LimitOption.LIMIT_OPTION_DEFAULT: 0> op=<SetOperation.SETOP_NONE: 0> all=False> stmt_location=0 stmt

In [27]:
json.loads(pglast.parser.parse_sql_json('SELECT * FROM review r WHERE r.i_id=100 ORDER BY creation_date DESC'))

{'version': 130003,
 'stmts': [{'stmt': {'SelectStmt': {'targetList': [{'ResTarget': {'val': {'ColumnRef': {'fields': [{'A_Star': {}}],
          'location': 7}},
        'location': 7}}],
     'fromClause': [{'RangeVar': {'relname': 'review',
        'inh': True,
        'relpersistence': 'p',
        'alias': {'aliasname': 'r'},
        'location': 14}}],
     'whereClause': {'A_Expr': {'kind': 'AEXPR_OP',
       'name': [{'String': {'str': '='}}],
       'lexpr': {'ColumnRef': {'fields': [{'String': {'str': 'r'}},
          {'String': {'str': 'i_id'}}],
         'location': 29}},
       'rexpr': {'A_Const': {'val': {'Integer': {'ival': 100}},
         'location': 36}},
       'location': 35}},
     'sortClause': [{'SortBy': {'node': {'ColumnRef': {'fields': [{'String': {'str': 'creation_date'}}],
          'location': 49}},
        'sortby_dir': 'SORTBY_DESC',
        'sortby_nulls': 'SORTBY_NULLS_DEFAULT',
        'location': -1}}],
     'limitOption': 'LIMIT_OPTION_DEFAULT',
     

In [45]:


pglast.parser.parse_sql_json('UPDATE review SET rating = $1 WHERE i_id=$2 AND u_id=$3')

'{"version":130003,"stmts":[{"stmt":{"UpdateStmt":{"relation":{"relname":"review","inh":true,"relpersistence":"p","location":7},"targetList":[{"ResTarget":{"name":"rating","val":{"ParamRef":{"number":1,"location":27}},"location":18}}],"whereClause":{"BoolExpr":{"boolop":"AND_EXPR","args":[{"A_Expr":{"kind":"AEXPR_OP","name":[{"String":{"str":"="}}],"lexpr":{"ColumnRef":{"fields":[{"String":{"str":"i_id"}}],"location":36}},"rexpr":{"ParamRef":{"number":2,"location":41}},"location":40}},{"A_Expr":{"kind":"AEXPR_OP","name":[{"String":{"str":"="}}],"lexpr":{"ColumnRef":{"fields":[{"String":{"str":"u_id"}}],"location":48}},"rexpr":{"ParamRef":{"number":3,"location":53}},"location":52}}],"location":44}}}}}]}'